In [ ]:
"""
統合人工プロモータライブラリ生成システム v4.0
=============================================

【構造】
[TFBM Module] - [Connector Spacer: 4/7/10 bp] - [Core Promoter: 34種類] - [Barcode: 20 bp]

【ライブラリサイズ】
総数 = 58 × 2 × 2 × 3 × 34 = 23,664 プロモータ
"""

import pandas as pd
import random
from datetime import datetime
from typing import Dict, List, Set


def calc_gc(seq: str) -> float:
    """GC含有率を計算"""
    return round((seq.count('G') + seq.count('C')) / len(seq) * 100, 1)


# ============================================================================
# Spacer Library
# ============================================================================

class SpacerLib:
    INTERNAL = {
        5: ['ACTGA', 'CAGTA', 'GTCAG', 'TACGT', 'AGTCA'],
        10: ['ACTGAGTCAG', 'CAGTACGTAG', 'GTCAGATCGA', 'TACGTAGTCA', 'AGTCATGCAG'],
    }
    CONNECTOR = {4: 'ACTG', 7: 'CAGTACG', 10: 'ACTGAGTCAG'}
    CORE = {5: 'ACTGA', 6: 'CAGTAC', 10: 'ACTGAGTCAG', 15: 'CAGTACGTAGTCAGT',
            18: 'ACTGAGTCAGACTGAGTC', 20: 'CAGTACGTAGTCAGTACGTA',
            24: 'ACTGAGTCAGACTGAGTCAGACTG', 27: 'CAGTACGTAGTCAGTACGTAGTCAGTC',
            28: 'ACTGAGTCAGACTGAGTCAGACTGAGTC'}

    @classmethod
    def internal(cls, length: int, idx: int = 0) -> str:
        return cls.INTERNAL[length][idx % len(cls.INTERNAL[length])]

    @classmethod
    def connector(cls, length: int) -> str:
        return cls.CONNECTOR.get(length, ('ACTGAGTCAG' * 3)[:length])

    @classmethod
    def core(cls, length: int) -> str:
        return cls.CORE.get(length, ('ACTGAGTCAG' * 3)[:length])

    @staticmethod
    def validate(seq: str) -> Dict:
        gc = calc_gc(seq)
        rev_comp = seq.translate(str.maketrans('ACGT', 'TGCA'))[::-1]
        return {
            'length': len(seq), 'gc_content': gc,
            'has_homopolymer': any(b * 4 in seq for b in 'ACGT'),
            'is_palindrome': seq == seq[::-1],
            'has_self_complementary': any(seq[i:i+6] in rev_comp for i in range(len(seq)-5)) if len(seq) >= 6 else False,
            'quality': 'PASS' if 40 <= gc <= 60 else 'WARN'
        }


# ============================================================================
# Barcode Generator
# ============================================================================

class BarcodeGenerator:
    def __init__(self, length: int = 20, min_hamming: int = 4, seed: int = 42):
        self.length, self.min_hamming = length, min_hamming
        self.barcodes: Set[str] = set()
        self.barcode_map: Dict[str, str] = {}
        random.seed(seed)

    def _valid(self, bc: str) -> bool:
        gc = (bc.count('G') + bc.count('C')) / len(bc)
        if not (0.4 <= gc <= 0.6): return False
        if any(b * 4 in bc for b in 'ACGT'): return False
        rev = ''.join({'A':'T','T':'A','G':'C','C':'G'}[b] for b in reversed(bc))
        if any(bc[i:i+8] in rev for i in range(len(bc)-7)): return False
        return all(sum(a != b for a, b in zip(bc, ex)) >= self.min_hamming for ex in self.barcodes)

    def generate(self, pid: str) -> str:
        if pid in self.barcode_map: return self.barcode_map[pid]
        for _ in range(10000):
            bc = ''.join(random.choices('ACGT', k=self.length))
            if self._valid(bc):
                self.barcodes.add(bc)
                self.barcode_map[pid] = bc
                return bc
        raise ValueError("Failed to generate barcode")


# ============================================================================
# TF Database (~58 TFs)
# ============================================================================

TF_DB = {
    # SP/KLF Family
    'SP1': ('GGGCGGG', 'SP/KLF', 1.8, 'MA0079.5'), 'SP2': ('GGGCGGG', 'SP/KLF', 1.4, 'MA0516.2'),
    'SP3': ('GGGCGGG', 'SP/KLF', 1.6, 'MA0746.2'), 'SP4': ('GGGCGGG', 'SP/KLF', 1.3, 'MA0685.1'),
    'KLF2': ('CACCCAC', 'SP/KLF', 1.4, 'MA0513.1'), 'KLF3': ('CACCCAC', 'SP/KLF', 1.3, 'MA0748.2'),
    'KLF4': ('CACCCAC', 'SP/KLF', 1.6, 'MA0039.4'), 'KLF5': ('CACCCAC', 'SP/KLF', 1.5, 'MA0599.1'),
    'KLF6': ('CACCCAC', 'SP/KLF', 1.3, 'MA0750.2'), 'KLF7': ('CACCCAC', 'SP/KLF', 1.3, 'MA0751.1'),
    'KLF9': ('CACCCAC', 'SP/KLF', 1.4, 'MA0517.1'), 'KLF10': ('CACCCAC', 'SP/KLF', 1.3, 'MA0752.1'),
    'KLF16': ('CACCCAC', 'SP/KLF', 1.3, 'MA0741.1'),
    # ETS Family
    'ETS1': ('ACCGGAAGT', 'ETS', 1.5, 'MA0098.3'), 'ETS2': ('ACCGGAAGT', 'ETS', 1.4, 'MA0624.2'),
    'GABPA': ('ACCGGAAGTG', 'ETS', 1.6, 'MA0062.3'), 'ELF1': ('CCGGAAGT', 'ETS', 1.6, 'MA0473.3'),
    'ELF2': ('CCGGAAGT', 'ETS', 1.4, 'MA0641.1'), 'ELF4': ('CCGGAAGT', 'ETS', 1.4, 'MA0641.1'),
    'ETV1': ('ACCGGAAGT', 'ETS', 1.4, 'MA0761.2'), 'ETV4': ('ACCGGAAGT', 'ETS', 1.5, 'MA0764.2'),
    'ETV5': ('ACCGGAAGT', 'ETS', 1.4, 'MA0765.1'), 'ETV6': ('ACCGGAAGT', 'ETS', 1.3, 'MA0645.1'),
    'ERG': ('ACGGAAGT', 'ETS', 1.5, 'MA0474.3'), 'FLI1': ('ACCGGAAGT', 'ETS', 1.5, 'MA0475.2'),
    # NF-Y Complex
    'NFYA': ('GATTGGCTGATTGG', 'NF-Y', 1.7, 'MA0060.3'), 'NFYB': ('GATTGGCTGATTGG', 'NF-Y', 1.6, 'MA0502.2'),
    'NFYC': ('GATTGGCTGATTGG', 'NF-Y', 1.5, 'MA0501.1'),
    # bHLH Family
    'USF1': ('GTCACGTGAC', 'bHLH', 1.6, 'MA0093.3'), 'USF2': ('GTCACGTGAC', 'bHLH', 1.6, 'MA0526.4'),
    'MAX': ('CACGTG', 'bHLH-LZ', 1.5, 'MA0058.3'), 'MYC': ('CACGTG', 'bHLH-LZ', 1.9, 'MA0147.3'),
    'MYCN': ('CACGTG', 'bHLH-LZ', 1.8, 'MA0104.4'), 'TFAP4': ('CAGCTG', 'bHLH', 1.4, 'MA0108.2'),
    'TCF3': ('CAGCTG', 'bHLH', 1.4, 'MA0522.3'), 'TCF12': ('CAGCTG', 'bHLH', 1.4, 'MA0521.1'),
    # E2F Family
    'E2F1': ('TTTCGCGCGAAA', 'E2F', 1.7, 'MA0024.3'), 'E2F2': ('TTTCGCGCGAAA', 'E2F', 1.5, 'MA0470.2'),
    'E2F3': ('TTTCGCGCGAAA', 'E2F', 1.5, 'MA0471.2'), 'E2F4': ('TTTCGCGCGAAA', 'E2F', 1.4, 'MA0541.2'),
    # Others
    'NRF1': ('GCGCATGCGC', 'NRF', 1.6, 'MA0506.2'),
    'YY1': ('CCGCCATATT', 'YY1', 1.5, 'MA0095.2'), 'YY2': ('CCGCCATATT', 'YY1', 1.3, 'MA0748.1'),
    'ZNF143': ('CCCAGCAG', 'ZNF', 1.7, 'MA0088.2'), 'ZNF148': ('GGGGGAGG', 'ZNF', 1.4, 'MA0752.1'),
    'MAZ': ('GGGAGGG', 'ZNF', 1.6, 'MA0056.2'), 'CTCF': ('CCGCGAGGGGGCAG', 'ZNF', 1.4, 'MA0139.1'),
    'TEAD1': ('GGAATG', 'TEAD', 1.4, 'MA0090.3'), 'TEAD2': ('GGAATG', 'TEAD', 1.4, 'MA0807.1'),
    'TEAD3': ('GGAATG', 'TEAD', 1.3, 'MA0808.1'), 'TEAD4': ('GGAATG', 'TEAD', 1.4, 'MA0809.2'),
    'TFAP2A': ('GCCGGGGGC', 'TFAP2', 1.5, 'MA0003.4'), 'TFAP2C': ('GCCGGGGGC', 'TFAP2', 1.5, 'MA0524.2'),
    'CEBPA': ('TTGCGCAA', 'CEBP', 1.5, 'MA0102.4'), 'CEBPB': ('TTGCGCAA', 'CEBP', 1.5, 'MA0466.3'),
    'CEBPG': ('TTGCGCAA', 'CEBP', 1.4, 'MA0838.1'),
    'RUNX1': ('TGTGGT', 'RUNX', 1.4, 'MA0002.2'), 'RUNX2': ('TGTGGT', 'RUNX', 1.4, 'MA0511.2'),
}


def get_tf_dataframe() -> pd.DataFrame:
    return pd.DataFrame([
        {'tf_name': k, 'consensus_sequence': v[0], 'family': v[1], 'activity_score': v[2], 'jaspar_id': v[3]}
        for k, v in TF_DB.items()
    ])


# ============================================================================
# TFBM Module Generator
# ============================================================================

def generate_tfbm_modules() -> pd.DataFrame:
    modules = []
    for tf, (cons, fam, act, jaspar) in TF_DB.items():
        for sp_len in [5, 10]:
            spacer = SpacerLib.internal(sp_len)
            for copies in [4, 6]:
                seq = spacer.join([cons] * copies)
                modules.append({
                    'tf_name': tf, 'tfbm_sequence': cons, 'copy_number': copies,
                    'internal_spacer_length': sp_len, 'internal_spacer': spacer,
                    'full_tfbm_module': seq, 'module_length': len(seq),
                    'tf_family': fam, 'activity_score': act, 'gc_content': calc_gc(seq), 'jaspar_id': jaspar
                })
    return pd.DataFrame(modules)


# ============================================================================
# Core Promoter Database (34 types)
# ============================================================================

def get_core_promoters() -> pd.DataFrame:
    sp = lambda n: SpacerLib.core(n)
    
    # Elements
    E = {
        'bre_opt': 'CGCGCC', 'bre_cons': 'GCCGCC',
        'tata_adml': 'TATAAAAG', 'tata_ext': 'TATAAAA', 'tata_cons': 'TATAAA', 'tata_sym': 'TATATATA',
        'inr_super': 'CTCAGTCTT', 'inr_adml': 'CTCAGTCT', 'inr_dual': 'CTCAGTCT', 'inr_weak': 'CTCAGT',
        'mte_strong': 'CTAACGGAACGG', 'mte_cons': 'CAAACGGAACGG',
        'dpe_ext': 'AGATCCCG', 'dpe_cons': 'AGATC',
        'dce_si': 'CTTC', 'dce_sii': 'CTGT', 'dce_siii': 'AGC',
        'sp1': 'GGGCGGG', 'ccaat': 'GATTGG', 'ets': 'GGAAGT', 'ebox': 'CACGTG',
        'nrf1': 'GCGCATGCGC', 'gc20': 'GCGCGCGCGCGCGCGCGCGC',
    }

    # (id, group, name, config, has_tata, tata_type, inr_type, has_mte, has_dpe, gc_rich, seq_parts, activity)
    DEFS = [
        # TATA+ (8)
        ('CP01','TATA+','Strongest_Full','BRE_opt + TATA_AdML + INR_super + MTE_strong + DPE_ext',True,'AdML','Super',True,True,False,
         [E['bre_opt'],sp(5),E['tata_adml'],sp(27),E['inr_super'],sp(18),E['mte_strong'],sp(10),E['dpe_ext']],'Very High'),
        ('CP02','TATA+','TATA_AdML_INR_super_MTE','TATA_AdML + INR_super + MTE_strong',True,'AdML','Super',True,False,False,
         [E['tata_adml'],sp(27),E['inr_super'],sp(18),E['mte_strong']],'High'),
        ('CP03','TATA+','BRE_TATA_AdML_set','BRE_opt + TATA_AdML + INR_AdML + MTE_cons',True,'AdML','AdML',True,False,False,
         [E['bre_opt'],sp(5),E['tata_adml'],sp(27),E['inr_adml'],sp(18),E['mte_cons']],'High'),
        ('CP04','TATA+','TATA_ext_full','TATA_ext + INR_super + MTE_strong + DPE_ext',True,'Extended','Super',True,True,False,
         [E['tata_ext'],sp(27),E['inr_super'],sp(18),E['mte_strong'],sp(10),E['dpe_ext']],'High'),
        ('CP05','TATA+','TATA_cons_basic','TATA_cons + INR_super + MTE_strong',True,'Consensus','Super',True,False,False,
         [E['tata_cons'],sp(27),E['inr_super'],sp(18),E['mte_strong']],'Medium-High'),
        ('CP06','TATA+','TATA_sym_variant','TATA_sym + INR_AdML + MTE_strong',True,'Symmetric','AdML',True,False,False,
         [E['tata_sym'],sp(27),E['inr_adml'],sp(18),E['mte_strong']],'Medium-High'),
        ('CP07','TATA+','All_consensus','BRE_cons + TATA_cons + INR_opt + MTE_cons',True,'Consensus','Optimized',True,False,False,
         [E['bre_cons'],sp(5),E['tata_cons'],sp(27),E['inr_adml'],sp(18),E['mte_cons']],'Medium'),
        ('CP08','TATA+','TATA_INR_minimal','TATA_AdML + INR_super (minimal)',True,'AdML','Super',False,False,False,
         [E['tata_adml'],sp(27),E['inr_super']],'Medium'),
        # TATA- (12)
        ('CP09','TATA-','INR_MTE_DPE_strong','INR_super + MTE_strong + DPE_ext',False,'None','Super',True,True,False,
         [E['inr_super'],sp(18),E['mte_strong'],sp(10),E['dpe_ext']],'High'),
        ('CP10','TATA-','Dual_INR_strong','Dual_INR + MTE_strong + DPE_ext',False,'None','Dual',True,True,False,
         [E['inr_dual'],sp(6),E['inr_dual'],sp(18),E['mte_strong'],sp(10),E['dpe_ext']],'High'),
        ('CP11','TATA-','Full_TATA_less','INR_super + MTE + DPE + DCE_full',False,'None','Super',True,True,False,
         [E['inr_super'],sp(18),E['mte_strong'],sp(10),E['dpe_ext'],sp(6),E['dce_si'],E['dce_sii'],E['dce_siii']],'High'),
        ('CP12','TATA-','INR_MTE_only','INR_super + MTE_strong',False,'None','Super',True,False,False,
         [E['inr_super'],sp(18),E['mte_strong']],'Medium-High'),
        ('CP13','TATA-','INR_DPE_only','INR_super + DPE_ext',False,'None','Super',False,True,False,
         [E['inr_super'],sp(28),E['dpe_ext']],'Medium'),
        ('CP14','TATA-','Triple_INR','Triple INR (max redundancy)',False,'None','Triple',False,False,False,
         [E['inr_super'],sp(6),E['inr_dual'],sp(6),E['inr_super']],'Medium-High'),
        ('CP15','TATA-','INR_consensus_elements','INR_super + MTE_cons + DPE_cons',False,'None','Super',True,True,False,
         [E['inr_super'],sp(18),E['mte_cons'],sp(10),E['dpe_cons']],'Medium'),
        ('CP16','TATA-','Dual_INR_MTE','Dual_INR + MTE_strong',False,'None','Dual',True,False,False,
         [E['inr_dual'],sp(6),E['inr_dual'],sp(18),E['mte_strong']],'Medium-High'),
        ('CP17','TATA-','INR_only_minimal','INR_super only (ultra-minimal)',False,'None','Super',False,False,False,
         [E['inr_super']],'Low-Medium'),
        ('CP18','TATA-','INR_MTE_DPE_DCE_partial','INR + MTE + DPE + DCE_SI+SII',False,'None','Super',True,True,False,
         [E['inr_super'],sp(18),E['mte_strong'],sp(10),E['dpe_ext'],sp(6),E['dce_si'],E['dce_sii']],'High'),
        ('CP19','TATA-','Dual_INR_DPE','Dual_INR + DPE_ext',False,'None','Dual',False,True,False,
         [E['inr_dual'],sp(6),E['inr_dual'],sp(24),E['dpe_ext']],'Medium'),
        ('CP20','TATA-','INR_DCE_only','INR_super + DCE_full',False,'None','Super',False,False,False,
         [E['inr_super'],sp(15),E['dce_si'],E['dce_sii'],E['dce_siii']],'Medium'),
        # Literature (6)
        ('CP21','Literature','AdML_Complete','Adenovirus Major Late Promoter',True,'AdML','AdML',True,False,False,
         [E['bre_opt'],sp(5),E['tata_adml'],sp(27),E['inr_adml'],sp(18),'CTAACGGAA'],'Very High'),
        ('CP22','Literature','SV40_Early','SV40 Early Promoter (minimal)',True,'Extended','Consensus',False,False,False,
         ['TATAAAA',sp(27),E['inr_adml']],'High'),
        ('CP23','Literature','CMV_Enhanced','CMV Minimal (enhanced version)',True,'CMV','Super',True,False,False,
         [E['tata_sym'],sp(27),E['inr_super'],sp(18),E['mte_strong']],'Very High'),
        ('CP24','Literature','HSP70_Core','Heat Shock Protein 70 Core Promoter',True,'Consensus','Consensus',False,False,False,
         [E['tata_cons'],sp(27),E['inr_adml']],'High'),
        ('CP25','Literature','Beta_Actin_Core','β-Actin Core Promoter (TATA-less)',False,'None','Super',True,True,False,
         [E['inr_super'],sp(18),E['mte_strong'],sp(10),E['dpe_ext']],'Very High'),
        ('CP26','Literature','UBC_Core','Ubiquitin C Core Promoter (TATA-less)',False,'None','Dual',True,False,False,
         [E['inr_dual'],sp(6),E['inr_dual'],sp(18),E['mte_strong']],'Very High'),
        # CpG-like (8)
        ('CpG01','CpG-like','SP1_Cluster','(SP1)×3 + weak_INR (GAPDH-like)',False,'None','Weak',False,False,True,
         [E['sp1'],sp(6),E['sp1'],sp(6),E['sp1'],sp(10),E['inr_weak']],'High (Housekeeping)'),
        ('CpG02','CpG-like','NFY_SP1_Combo','CCAAT + (SP1)×2 + weak_INR',False,'None','Weak',False,False,True,
         [E['ccaat'],sp(10),E['sp1'],sp(6),E['sp1'],sp(10),E['inr_weak']],'High (Housekeeping)'),
        ('CpG03','CpG-like','ETS_SP1_Combo','ETS + (SP1)×2 + weak_INR',False,'None','Weak',False,False,True,
         [E['ets'],sp(10),E['sp1'],sp(6),E['sp1'],sp(10),E['inr_weak']],'High'),
        ('CpG04','CpG-like','Triple_Element','SP1 + CCAAT + SP1 + ETS + weak_INR',False,'None','Weak',False,False,True,
         [E['sp1'],sp(6),E['ccaat'],sp(6),E['sp1'],sp(6),E['ets'],sp(10),E['inr_weak']],'Very High'),
        ('CpG05','CpG-like','Ebox_SP1','E-box + (SP1)×2 + weak_INR',False,'None','Weak',False,False,True,
         [E['ebox'],sp(10),E['sp1'],sp(6),E['sp1'],sp(10),E['inr_weak']],'High'),
        ('CpG06','CpG-like','NRF1_SP1','NRF1 + (SP1)×2 + weak_INR (Mitochondrial)',False,'None','Weak',False,False,True,
         [E['nrf1'],sp(10),E['sp1'],sp(6),E['sp1'],sp(10),E['inr_weak']],'High (Mitochondrial)'),
        ('CpG07','CpG-like','Pure_GC_Stretch','GC-rich(20bp) + (SP1)×2 + weak_INR',False,'None','Weak',False,False,True,
         [E['gc20'],sp(6),E['sp1'],sp(6),E['sp1'],sp(10),E['inr_weak']],'High (CpG island)'),
        ('CpG08','CpG-like','Minimal_CpG','(SP1)×2 + CCAAT + SP1 (no INR)',False,'None','None',False,False,True,
         [E['sp1'],sp(6),E['sp1'],sp(6),E['ccaat'],sp(6),E['sp1']],'Medium-High (Dispersed)'),
    ]
    
    promoters = []
    for d in DEFS:
        seq = ''.join(d[10])
        promoters.append({
            'core_id': d[0], 'group': d[1], 'name': d[2], 'config': d[3],
            'has_tata': d[4], 'tata_type': d[5], 'inr_type': d[6],
            'has_mte': d[7], 'has_dpe': d[8], 'gc_rich': d[9],
            'sequence': seq, 'length': len(seq), 'gc_content': calc_gc(seq), 'expected_activity': d[11]
        })
    return pd.DataFrame(promoters)


# ============================================================================
# Integrated Library Generator
# ============================================================================

class PromoterLibraryV4:
    def __init__(self, connector_lengths: List[int] = [4, 7, 10], seed: int = 42):
        self.connector_lengths = connector_lengths
        self.barcode_gen = BarcodeGenerator(seed=seed)

    def generate(self, verbose: bool = True) -> pd.DataFrame:
        if verbose:
            print("\n" + "="*80)
            print("🧬 統合人工プロモータライブラリ生成システム v4.0")
            print("="*80)
            print(f"実行時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            print("="*80)

        tfbm_df = generate_tfbm_modules()
        core_df = get_core_promoters()
        
        if verbose:
            print(f"\n📦 TFBMモジュール: {len(tfbm_df)} 生成完了")
            print(f"🎯 Core Promoter: {len(core_df)} 準備完了")
            total = len(tfbm_df) * len(core_df) * len(self.connector_lengths)
            print(f"🔗 統合プロモータ生成中... (予想: {total:,})")

        promoters = []
        for _, tfbm in tfbm_df.iterrows():
            for _, core in core_df.iterrows():
                for conn_len in self.connector_lengths:
                    pid = f"{tfbm['tf_name']}_{tfbm['copy_number']}x_int{tfbm['internal_spacer_length']}bp_con{conn_len}bp_{core['core_id']}"
                    connector = SpacerLib.connector(conn_len)
                    barcode = self.barcode_gen.generate(pid)
                    full_seq = tfbm['full_tfbm_module'] + connector + core['sequence'] + barcode
                    
                    promoters.append({
                        'promoter_id': pid, 'tf_name': tfbm['tf_name'], 'tf_family': tfbm['tf_family'],
                        'tfbm_sequence': tfbm['tfbm_sequence'], 'copy_number': tfbm['copy_number'],
                        'internal_spacer_length': tfbm['internal_spacer_length'], 'internal_spacer': tfbm['internal_spacer'],
                        'tfbm_module': tfbm['full_tfbm_module'], 'tfbm_module_length': tfbm['module_length'],
                        'tf_activity_score': tfbm['activity_score'], 'jaspar_id': tfbm['jaspar_id'],
                        'connector_spacer_length': conn_len, 'connector_spacer': connector,
                        'core_promoter_id': core['core_id'], 'core_promoter_name': core['name'],
                        'core_group': core['group'], 'core_config': core['config'],
                        'has_tata': core['has_tata'], 'tata_type': core['tata_type'], 'inr_type': core['inr_type'],
                        'has_mte': core['has_mte'], 'has_dpe': core['has_dpe'], 'gc_rich': core['gc_rich'],
                        'core_sequence': core['sequence'], 'core_length': core['length'],
                        'expected_activity': core['expected_activity'], 'barcode': barcode,
                        'full_sequence': full_seq, 'total_length': len(full_seq), 'gc_content': calc_gc(full_seq),
                    })

        df = pd.DataFrame(promoters)
        if verbose:
            self._stats(df)
        return df

    def _stats(self, df: pd.DataFrame):
        print("\n" + "="*80 + "\n📊 ライブラリ統計\n" + "="*80)
        print(f"\n【基本情報】\n  総プロモータ数: {len(df):,}\n  ユニークTF数: {df['tf_name'].nunique()}\n  Core Promoter種類: {df['core_promoter_id'].nunique()}")
        print("\n【TFファミリー分布】")
        for fam, cnt in df.groupby('tf_family').size().sort_values(ascending=False).items():
            print(f"  {fam}: {cnt:,}")
        print("\n【Core Promoterグループ分布】")
        for grp in ['TATA+', 'TATA-', 'Literature', 'CpG-like']:
            print(f"  {grp}: {len(df[df['core_group'] == grp]):,}")
        print(f"\n【Spacer設定】\n  内部spacer: {df['internal_spacer_length'].unique().tolist()} bp\n  コネクターspacer: {df['connector_spacer_length'].unique().tolist()} bp")
        print(f"\n【配列統計】\n  配列長範囲: {df['total_length'].min()} - {df['total_length'].max()} bp\n  平均配列長: {df['total_length'].mean():.1f} bp")
        print(f"  GC含量範囲: {df['gc_content'].min():.1f}% - {df['gc_content'].max():.1f}%\n  平均GC含量: {df['gc_content'].mean():.1f}%")
        print("="*80 + "\n")

    def export(self, df: pd.DataFrame, prefix: str = 'synthetic_promoter_library_v4'):
        print("\n📁 ファイルエクスポート中...")
        df.to_csv(f"{prefix}_complete.csv", index=False)
        df[['promoter_id','barcode','tf_name','core_group','core_promoter_id']].to_csv(f"{prefix}_barcode_reference.csv", index=False)
        df[['tf_name','tf_family','tfbm_sequence','tf_activity_score','jaspar_id']].drop_duplicates().to_csv(f"{prefix}_tf_info.csv", index=False)
        core_cols = ['core_promoter_id','core_promoter_name','core_group','core_config','has_tata','tata_type','inr_type','has_mte','has_dpe','gc_rich','core_sequence','core_length','expected_activity']
        df[core_cols].drop_duplicates().to_csv(f"{prefix}_core_promoter_info.csv", index=False)
        with open(f"{prefix}_sequences.fasta", 'w') as f:
            for _, r in df.iterrows():
                f.write(f">{r['promoter_id']}\n{r['full_sequence']}\n")
        with open(f"{prefix}_summary.txt", 'w') as f:
            f.write(f"Synthetic Promoter Library v4.0\nGenerated: {datetime.now()}\nTotal: {len(df):,} promoters\n")
            for fam, cnt in df.groupby('tf_family').size().items():
                f.write(f"  {fam}: {cnt}\n")
        print("✅ エクスポート完了！\n")


class NGSRecommendations:
    @staticmethod
    def recommend(library_size: int, reads_per_bc: int = 100):
        required = int(library_size * reads_per_bc / 0.8 * 1.5)
        print(f"\n{'='*80}\n🔬 NGSプラットフォーム推奨\n{'='*80}")
        print(f"ライブラリサイズ: {library_size:,}\n必要リード数: {required:,}\n")
        platforms = [("MiSeq v2", 15e6, "$1,500-3,000"), ("NextSeq Mid", 130e6, "$3,000-5,000"),
                     ("NextSeq High", 400e6, "$5,000-8,000"), ("NovaSeq S1", 1.6e9, "$8,000-12,000")]
        for name, reads, cost in platforms:
            if required <= reads:
                print(f"🎯 推奨: {name} (~{int(reads):,} reads, {cost})")
                break
        print("⚠️  PhiX 10-20%推奨, バーコードのみ: SE 75bp\n" + "="*80)


def main():
    lib = PromoterLibraryV4(connector_lengths=[4, 7, 10], seed=42)
    df = lib.generate(verbose=True)
    lib.export(df)
    NGSRecommendations.recommend(len(df))
    print(f"\n✅ {len(df):,} 種類の人工プロモータを生成しました\n")
    return df


if __name__ == "__main__":
    df = main()
